In [1]:
import pandas as  pd

DATASET_PATH = 'dataset/dataset_with_proper_sentiment.json'

In [2]:
df = pd.read_json(DATASET_PATH)
print(df.sentiment.value_counts())
df.head(15)

sentiment
neutral     32224
negative    28654
positive    25773
Name: count, dtype: int64


,text,drivers,sentiment
0,Fail of the day: Stroll. AKA Latifi & Magnusse...,[Stroll],negative
1,This why Ricc career is over... we see again t...,[Daniel Riccardo],negative
2,0:19,[],neutral
3,Hall Cynthia Jackson Kimberly Lewis Sharon,[],neutral
4,🔥,[],positive
5,Lee Ronald Hall Brenda Moore Jeffrey,[],neutral
6,"here after singapore gp 2024, oh how the table...",[],neutral
7,Where's the scream of the engine gone? ❤,[],neutral
8,Miller Angela Lewis Donald Davis Jason,[],neutral
9,"I'm a Red Bull fan, so when the FIA says nothi...",[Max Verstappen],neutral


In [3]:
# afin sentiment analysis

from afinn import Afinn

afinn = Afinn()

# add a column to the dataframe with the sentiment score

def map_score(score):
    if score > 0:
        return 'positive'
    elif score < 0:
        return 'negative'
    else:
        return 'neutral'

df['afinn_score'] = df['text'].apply(lambda x: map_score(afinn.score(x)))
print(df.afinn_score.value_counts())
df.head(15)

afinn_score
neutral     34249
positive    32553
negative    19849
Name: count, dtype: int64


,text,drivers,sentiment,afinn_score
0,Fail of the day: Stroll. AKA Latifi & Magnusse...,[Stroll],negative,negative
1,This why Ricc career is over... we see again t...,[Daniel Riccardo],negative,positive
2,0:19,[],neutral,neutral
3,Hall Cynthia Jackson Kimberly Lewis Sharon,[],neutral,neutral
4,🔥,[],positive,neutral
5,Lee Ronald Hall Brenda Moore Jeffrey,[],neutral,neutral
6,"here after singapore gp 2024, oh how the table...",[],neutral,neutral
7,Where's the scream of the engine gone? ❤,[],neutral,negative
8,Miller Angela Lewis Donald Davis Jason,[],neutral,neutral
9,"I'm a Red Bull fan, so when the FIA says nothi...",[Max Verstappen],neutral,positive


In [5]:
from sklearn.metrics import accuracy_score

accuracy_score(df['sentiment'], df['afinn_score'])

0.5799471442914681

In [7]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/kacper/nltk_data...


True

In [10]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

vader = SentimentIntensityAnalyzer()

def map_score_vader(score):
    if score['compound'] > 0.01:
        return 'positive'
    elif score['compound'] < -0.01:
        return 'negative'
    else:
        return 'neutral'
    
df['vader_score'] = df['text'].apply(lambda x: map_score_vader(vader.polarity_scores(x)))
print(df.vader_score.value_counts())
print(accuracy_score(df['sentiment'], df['vader_score']))
df.head(15)

vader_score
positive    34886
neutral     30598
negative    21167
Name: count, dtype: int64
0.5854866071943774


,text,drivers,sentiment,afinn_score,vader_score
0,Fail of the day: Stroll. AKA Latifi & Magnusse...,[Stroll],negative,negative,negative
1,This why Ricc career is over... we see again t...,[Daniel Riccardo],negative,positive,positive
2,0:19,[],neutral,neutral,neutral
3,Hall Cynthia Jackson Kimberly Lewis Sharon,[],neutral,neutral,neutral
4,🔥,[],positive,neutral,neutral
5,Lee Ronald Hall Brenda Moore Jeffrey,[],neutral,neutral,neutral
6,"here after singapore gp 2024, oh how the table...",[],neutral,neutral,neutral
7,Where's the scream of the engine gone? ❤,[],neutral,negative,negative
8,Miller Angela Lewis Donald Davis Jason,[],neutral,neutral,neutral
9,"I'm a Red Bull fan, so when the FIA says nothi...",[Max Verstappen],neutral,positive,positive


In [ ]:
# Porownac roznice i recznie sprawdic labele